In [1]:
import time
import numpy as np
from copy import deepcopy

from sorting import hybrid_sort, merge_sort
from comparison_counter import ComparisonCounter

np.random.seed(13)

# Testing the sorting algorithm

In [2]:
arr1 = [14, 40, 31, 28, 3, 15, 17, 51]
arr2 = [23, 23, 23, 23, 23, 23, 23, 23]

In [3]:
comparison_counter = ComparisonCounter()

print(merge_sort(deepcopy(arr1), comparison_counter))
assert comparison_counter.get_count() == 16

comparison_counter.reset_count()
print(merge_sort(deepcopy(arr2), comparison_counter))
assert comparison_counter.get_count() == 12

[3, 14, 15, 17, 28, 31, 40, 51]
[23, 23, 23, 23, 23, 23, 23, 23]


In [4]:
comparison_counter.reset_count()
print(hybrid_sort(deepcopy(arr1), comparison_counter, 4))
assert comparison_counter.get_count() >= 16

comparison_counter.reset_count()
print(hybrid_sort(deepcopy(arr2), comparison_counter, 4))
assert comparison_counter.get_count() == 10

[3, 14, 15, 17, 28, 31, 40, 51]
[23, 23, 23, 23, 23, 23, 23, 23]


# Input generation

In [6]:
sizes = [1000, 10000, 100000, 1000000, 10000000]
inputs = [np.random.randint(1, size + 1, size) for size in sizes]

for i in range(len(inputs)):
    print("Size: ", len(inputs[i]))
    print("Min value:", min(inputs[i]))
    print("Max value:", max(inputs[i]))
    print()

Size:  1000
Min value: 1
Max value: 999

Size:  10000
Min value: 1
Max value: 10000

Size:  100000
Min value: 1
Max value: 100000

Size:  1000000
Min value: 1
Max value: 1000000

Size:  10000000
Min value: 2
Max value: 9999999

